In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from gurobipy import *
from matplotlib import pyplot as plt
import collections
from copy import *
from IPython.display import HTML
from IPython.display import display
from collections import *

In [3]:
sample_dcs = ['A', 'B', 'C']

In [26]:
N = 10000
#T = 1000
    
demand_scenarios = []

for n in range(N):
    sample = []
    for i in sample_dcs:
        count = np.random.normal(100, 50)
        sample += int(count) * [i]
    demand_scenarios.append(sample)  

In [27]:
def optimize_revenue(S, D, y): 
    model = Model('model')
    model.setParam("LogToConsole", 0)

    fulfill = model.addVars(((i, j) for i in sample_dcs for j in sample_dcs), vtype=GRB.CONTINUOUS, name='fulfill')

    model.setObjective(sum(fulfill[i, j] for i, j in fulfill), GRB.MAXIMIZE)

    for i in sample_dcs:
        model.addConstr(fulfill.sum(i, '*') <= S[i], name='supply_%s' % (i))

    for j in sample_dcs:
        model.addConstr(fulfill.sum('*', j) <= D[j], name='demand_%s' % (j))
        
    for i in sample_dcs:
        for j in sample_dcs:
            model.addConstr(fulfill[i, j] <= y[i, j] * S[i], name='arc_%s_%s' % (i, j))

    model.optimize()
    obj = model.objVal
    
    return model, fulfill, obj


In [28]:
def solve_allocation_gradient(S, y):
    solutions_y = collections.defaultdict(float)
    solutions_y_round = []
    solutions_obj_round = []
    M = 1000
    for _iter in range(M):
        ## rounding to integer solution
        y_round = {}
        for i in sample_dcs:
            for j in sample_dcs:
                mu = y[i, j]
                y_round[i, j] = np.random.choice(a=[0, 1], size=1, p=[1-mu, mu])[0]
            
        solutions_y_round.append(y_round)
        
        ## generate one sample
        sample_path = demand_scenarios[_iter]
        D = collections.defaultdict(float)
        for i in sample_path:
            D[i] += 1 
        
        ## run with integer
        model, fulfill, revenue = optimize_revenue(S,  D, y_round)
        
        fix_cost = sum([b[i, j] * y_round[i, j] for i in sample_dcs for j in sample_dcs])
        
        solutions_obj_round.append(revenue - fix_cost)      
           
        ## run with fraction 
        model, fulfill, obj = optimize_revenue(S, D, y)
        _fulfill = model.getAttr('X', fulfill)
        
        print(obj)

        ## update y
        eta = 0.1 / (np.sqrt(_iter + 1))
        #eta = 1 / np.sqrt(100)
        dual, grad = {}, {}   
        for i in sample_dcs:
            for j in sample_dcs:
                name = 'arc_' + str(i) + '_' + str(j)
                dual[i, j] = model.getConstrByName(name).Pi
                grad[i, j] = dual[i, j] * S[i] - b[i, j] 
                
                y[i, j] = min(1, max(0, y[i, j] + eta * grad[i, j]))
                solutions_y[i, j] += y[i, j]
            
        print(y)
        
    for i in sample_dcs:
        for j in sample_dcs:
            y[i, j] = solutions_y[i, j] / M
            
    return solutions_y_round, solutions_obj_round

In [29]:
b, y, S = collections.defaultdict(), collections.defaultdict(), collections.defaultdict()

for i in sample_dcs:
    for j in sample_dcs:
        if i == j:
            b[i, j] = 5
        else:
            b[i, j] = 6

for i in sample_dcs:
    for j in sample_dcs:
        y[i, j] = 0
    
for i in sample_dcs:
    S[i] = 100
    
solutions_y_round, solutions_obj_round = solve_allocation_gradient(S, y)

-0.0
defaultdict(None, {('A', 'A'): 1, ('A', 'B'): 1, ('A', 'C'): 1, ('B', 'A'): 1, ('B', 'B'): 1, ('B', 'C'): 1, ('C', 'A'): 1, ('C', 'B'): 1, ('C', 'C'): 1})
212.0
defaultdict(None, {('A', 'A'): 0.6464466094067263, ('A', 'B'): 0.5757359312880714, ('A', 'C'): 0.5757359312880714, ('B', 'A'): 0.5757359312880714, ('B', 'B'): 0.6464466094067263, ('B', 'C'): 0.5757359312880714, ('C', 'A'): 0.5757359312880714, ('C', 'B'): 0.5757359312880714, ('C', 'C'): 0.6464466094067263})
300.0
defaultdict(None, {('A', 'A'): 0.35777147481191335, ('A', 'B'): 0.22932576977429597, ('A', 'C'): 0.22932576977429597, ('B', 'A'): 0.22932576977429597, ('B', 'B'): 0.35777147481191335, ('B', 'C'): 0.22932576977429597, ('C', 'A'): 0.22932576977429597, ('C', 'B'): 0.22932576977429597, ('C', 'C'): 0.35777147481191335})
174.28460287210106
defaultdict(None, {('A', 'A'): 0.10777147481191335, ('A', 'B'): 1, ('A', 'C'): 1, ('B', 'A'): 0, ('B', 'B'): 1, ('B', 'C'): 1, ('C', 'A'): 0, ('C', 'B'): 1, ('C', 'C'): 1})
140.7771474

300.0
defaultdict(None, {('A', 'A'): 0.9444444444444444, ('A', 'B'): 0.7308093979041025, ('A', 'C'): 0.9333333333333333, ('B', 'A'): 0.9333333333333333, ('B', 'B'): 0.7756744982534187, ('B', 'C'): 0.9333333333333333, ('C', 'A'): 0.9333333333333333, ('C', 'B'): 0.7308093979041025, ('C', 'C'): 0.9444444444444444})
300.0
defaultdict(None, {('A', 'A'): 0.8892286814070212, ('A', 'B'): 0.6645504822591946, ('A', 'C'): 0.8670744176884254, ('B', 'A'): 0.8670744176884254, ('B', 'B'): 0.7204587352159955, ('B', 'C'): 0.8670744176884254, ('C', 'A'): 0.8670744176884254, ('C', 'B'): 0.6645504822591946, ('C', 'C'): 0.8892286814070212})
300.0
defaultdict(None, {('A', 'A'): 0.834346551412176, ('A', 'B'): 0.5986919262653804, ('A', 'C'): 0.8012158616946112, ('B', 'A'): 0.8012158616946112, ('B', 'B'): 0.6655766052211503, ('B', 'C'): 0.8012158616946112, ('C', 'A'): 0.8012158616946112, ('C', 'B'): 0.5986919262653804, ('C', 'C'): 0.834346551412176})
254.0
defaultdict(None, {('A', 'A'): 0.7797920788531778, ('A

300.0
defaultdict(None, {('A', 'A'): 0.6792030997288467, ('A', 'B'): 0.677685777939565, ('A', 'C'): 0.9046834678063675, ('B', 'A'): 0.6150437196746158, ('B', 'B'): 0.8004722073855286, ('B', 'C'): 0.9046834678063675, ('C', 'A'): 0.6150437196746158, ('C', 'B'): 0.677685777939565, ('C', 'C'): 0.9205695565053063})
300.0
defaultdict(None, {('A', 'A'): 0.639674628976742, ('A', 'B'): 0.6302516130370392, ('A', 'C'): 0.8572493029038418, ('B', 'A'): 0.5676095547720901, ('B', 'B'): 0.7609437366334239, ('B', 'C'): 0.8572493029038418, ('C', 'A'): 0.5676095547720901, ('C', 'B'): 0.6302516130370392, ('C', 'C'): 0.8810410857532015})
300.0
defaultdict(None, {('A', 'A'): 0.600269108664787, ('A', 'B'): 0.5829649886626932, ('A', 'C'): 0.8099626785294958, ('B', 'A'): 0.520322930397744, ('B', 'B'): 0.7215382163214689, ('B', 'C'): 0.8099626785294958, ('C', 'A'): 0.520322930397744, ('C', 'B'): 0.5829649886626932, ('C', 'C'): 0.8416355654412465})
300.0
defaultdict(None, {('A', 'A'): 0.5609853985988676, ('A', '

193.0
defaultdict(None, {('A', 'A'): 0.8057390920480099, ('A', 'B'): 0.5572148513266648, ('A', 'C'): 0.15556025538072948, ('B', 'A'): 0.5973890180496335, ('B', 'B'): 0.641956383805367, ('B', 'C'): 0.15556025538072948, ('C', 'A'): 0.5973890180496335, ('C', 'B'): 0.5572148513266648, ('C', 'C'): 0.4430541170109783})
216.0
defaultdict(None, {('A', 'A'): 0.7735978747213486, ('A', 'B'): 0.5186453905346713, ('A', 'C'): 0.11699079458873599, ('B', 'A'): 0.55881955725764, ('B', 'B'): 0.6098151664787057, ('B', 'C'): 0.11699079458873599, ('C', 'A'): 0.55881955725764, ('C', 'B'): 0.5186453905346713, ('C', 'C'): 0.41091289968431705})
201.4894488861789
defaultdict(None, {('A', 'A'): 0.7415228597663693, ('A', 'B'): 0.4801553725886962, ('A', 'C'): 0.7200010757423452, ('B', 'A'): 0.5203295393116649, ('B', 'B'): 0.5777401515237265, ('B', 'C'): 0.7200010757423452, ('C', 'A'): 0.5203295393116649, ('C', 'B'): 0.4801553725886962, ('C', 'C'): 1})
275.0
defaultdict(None, {('A', 'A'): 0.7095136397831453, ('A', 

260.15619999009283
defaultdict(None, {('A', 'A'): 0.605256964507513, ('A', 'B'): 0.9815545037755085, ('A', 'C'): 0.8798938628286392, ('B', 'A'): 0.5263083574090157, ('B', 'B'): 1, ('B', 'C'): 0.8798938628286392, ('C', 'A'): 0.5263083574090157, ('C', 'B'): 0.6519064502344033, ('C', 'C'): 0.9441418716719552})
300.0
defaultdict(None, {('A', 'A'): 0.5773930538787453, ('A', 'B'): 0.9481178110209872, ('A', 'C'): 0.846457170074118, ('B', 'A'): 0.49287166465449456, ('B', 'B'): 0.9721360893712323, ('B', 'C'): 0.846457170074118, ('C', 'A'): 0.49287166465449456, ('C', 'B'): 0.6184697574798821, ('C', 'C'): 0.9162779610431875})
256.0
defaultdict(None, {('A', 'A'): 0.5495723096750125, ('A', 'B'): 0.9147329179765078, ('A', 'C'): 0.8130722770296386, ('B', 'A'): 0.4594867716100151, ('B', 'B'): 0.9443153451674995, ('B', 'C'): 0.8130722770296386, ('C', 'A'): 0.4594867716100151, ('C', 'B'): 0.5850848644354026, ('C', 'C'): 0.8884572168394547})
300.0
defaultdict(None, {('A', 'A'): 0.5217945318972347, ('A', 

...

196.9326715266147
defaultdict(None, {('A', 'A'): 0.997103385898817, ('A', 'B'): 0.5002154913714394, ('A', 'C'): 0.4227696054000514, ('B', 'A'): 0.5783293655484751, ('B', 'B'): 0.8587555934683269, ('B', 'C'): 0.4242215137922576, ('C', 'A'): 0.5522075005518146, ('C', 'B'): 0.6394620845297814, ('C', 'C'): 0.7244780087641538})
159.0
defaultdict(None, {('A', 'A'): 0.9775368738939462, ('A', 'B'): 0.47673567696559443, ('A', 'C'): 0.3992897909942064, ('B', 'A'): 0.5548495511426301, ('B', 'B'): 0.839189081463456, ('B', 'C'): 0.40074169938641263, ('C', 'A'): 0.5287276861459695, ('C', 'B'): 0.6159822701239364, ('C', 'C'): 0.704911496759283})
300.0
defaultdict(None, {('A', 'A'): 0.9579853267188018, ('A', 'B'): 0.4532738203554212, ('A', 'C'): 0.37582793438403317, ('B', 'A'): 0.5313876945324568, ('B', 'B'): 0.8196375342883117, ('B', 'C'): 0.3772798427762394, ('C', 'A'): 0.5052658295357962, ('C', 'B'): 0.5925204135137631, ('C', 'C'): 0.6853599495841386})
297.0
defaultdict(None, {('A', 'A'): 0.9384487

300.0
defaultdict(None, {('A', 'A'): 0.7375352179161078, ('A', 'B'): 0.1737358490818144, ('A', 'C'): 0.4703153049240842, ('B', 'A'): 0.2378286659993715, ('B', 'B'): 0.3270202057017251, ('B', 'C'): 0.4786840260668871, ('C', 'A'): 0.2243088658685837, ('C', 'B'): 0.31272390735212036, ('C', 'C'): 0.8333325492887965})
262.347996213566
defaultdict(None, {('A', 'A'): 0.7190798811529687, ('A', 'B'): 0.5206961802288286, ('A', 'C'): 0.44816890080831734, ('B', 'A'): 0.21568226188360462, ('B', 'B'): 0.6776716042013671, ('B', 'C'): 0.45653762195112024, ('C', 'A'): 0.20216246175281682, ('C', 'B'): 0.6596842384991346, ('C', 'C'): 0.8148772125256575})
299.692460478939
defaultdict(None, {('A', 'A'): 1, ('A', 'B'): 0.49856484682192903, ('A', 'C'): 0.4260375674014178, ('B', 'A'): 0.5624064852583639, ('B', 'B'): 0.6592288263622841, ('B', 'C'): 0.4344062885442207, ('C', 'A'): 0.5488866851275761, ('C', 'B'): 0.6375529050922351, ('C', 'C'): 0.7964344346865745})
300.0
defaultdict(None, {('A', 'A'): 0.98156975

293.0
defaultdict(None, {('A', 'A'): 0.6746358930286259, ('A', 'B'): 0.5103441357777319, ('A', 'C'): 0.44373297894720237, ('B', 'A'): 0.2186792363885291, ('B', 'B'): 0.8938342237602429, ('B', 'C'): 0.4521017000900051, ('C', 'A'): 0.5657686592450506, ('C', 'B'): 0.6450362344509337, ('C', 'C'): 0.68735906884508})
174.0
defaultdict(None, {('A', 'A'): 0.6569802792399647, ('A', 'B'): 0.4891573992313384, ('A', 'C'): 0.4225462424008089, ('B', 'A'): 0.19749249984213563, ('B', 'B'): 0.8761786099715817, ('B', 'C'): 0.43091496354361164, ('C', 'A'): 0.5445819226986571, ('C', 'B'): 0.6238494979045403, ('C', 'C'): 0.6697034550564188})
194.0
defaultdict(None, {('A', 'A'): 0.6393356624089501, ('A', 'B'): 0.4679838590341208, ('A', 'C'): 0.40137270220359134, ('B', 'A'): 0.17631895964491806, ('B', 'B'): 0.8585339931405671, ('B', 'C'): 0.4097414233463941, ('C', 'A'): 0.5234083825014395, ('C', 'B'): 0.6026759577073226, ('C', 'C'): 0.6520588382254042})
300.0
defaultdict(None, {('A', 'A'): 0.6217020220124851

300.0
defaultdict(None, {('A', 'A'): 0.9667128629037063, ('A', 'B'): 0.4701472193816751, ('A', 'C'): 0.4056617452677597, ('B', 'A'): 0.5185456581797534, ('B', 'B'): 0.7542419160371824, ('B', 'C'): 0.4116049552798022, ('C', 'A'): 0.5224321040128832, ('C', 'B'): 0.26042102289573166, ('C', 'C'): 0.6543698042250445})
283.0
defaultdict(None, {('A', 'A'): 0.9500831102727628, ('A', 'B'): 0.4501915162245429, ('A', 'C'): 0.38570604211062753, ('B', 'A'): 0.4985899550226212, ('B', 'B'): 0.7376121634062389, ('B', 'C'): 0.39164925212267, ('C', 'A'): 0.5024764008557511, ('C', 'B'): 0.24046531973859947, ('C', 'C'): 0.6377400515941011})
105.0
defaultdict(None, {('A', 'A'): 0.9334625478898995, ('A', 'B'): 0.4302468413651069, ('A', 'C'): 0.3657613672511915, ('B', 'A'): 0.4786452801631852, ('B', 'B'): 0.7209916010233756, ('B', 'C'): 0.37170457726323397, ('C', 'A'): 0.48253172599631505, ('C', 'B'): 0.22052064487916345, ('C', 'C'): 0.6211194892112377})
300.0
defaultdict(None, {('A', 'A'): 0.916851160535272

286.14279648721526
defaultdict(None, {('A', 'A'): 0.6855818467626815, ('A', 'B'): 0.6580744613234317, ('A', 'C'): 0.3033836954529143, ('B', 'A'): 0.38019431107547746, ('B', 'B'): 0.9494359753063351, ('B', 'C'): 0.6012655502690252, ('C', 'A'): 0.38408075690860727, ('C', 'B'): 0.44834826483748813, ('C', 'C'): 0.8676771626441231})
300.0
defaultdict(None, {('A', 'A'): 0.669452814504617, ('A', 'B'): 0.6387196226137543, ('A', 'C'): 0.2840288567432369, ('B', 'A'): 0.36083947236580005, ('B', 'B'): 0.9333069430482706, ('B', 'C'): 0.5819107115593477, ('C', 'A'): 0.36472591819892985, ('C', 'B'): 0.4289934261278107, ('C', 'C'): 0.8515481303860586})
131.0
defaultdict(None, {('A', 'A'): 0.653332167499138, ('A', 'B'): 0.6193748462071794, ('A', 'C'): 0.2646840803366621, ('B', 'A'): 0.3414946959592252, ('B', 'B'): 0.9171862960427916, ('B', 'C'): 0.5625659351527729, ('C', 'A'): 0.345381141792355, ('C', 'B'): 0.4096486497212359, ('C', 'C'): 0.8354274833805796})
144.02080052507182
defaultdict(None, {('A',

In [30]:
total_obj = collections.defaultdict(float)
total_round = collections.defaultdict(int)

for n in range(1000):
    y = solutions_y_round[n]
    y_tuple = tuple([y[i, j] for i in sample_dcs for j in sample_dcs])
    obj = solutions_obj_round[n]
    total_obj[y_tuple] += obj
    total_round[y_tuple] += 1

In [31]:
summary = collections.defaultdict(float)

for key in total_round:
    average_revenue = total_obj[key] / total_round[key]
    summary[key] = average_revenue

In [32]:
{k: v for k, v in sorted(summary.items(), key=lambda item: item[1])}

{(0, 0, 0, 0, 0, 0, 0, 0, 0): 0.0,
 (0, 0, 0, 0, 1, 0, 0, 0, 0): 25.0,
 (0, 0, 1, 0, 0, 0, 0, 0, 1): 31.0,
 (1, 0, 0, 0, 0, 0, 1, 0, 0): 45.0,
 (0, 0, 0, 1, 1, 0, 1, 0, 0): 61.0,
 (0, 1, 1, 1, 1, 1, 1, 0, 0): 73.0,
 (0, 0, 0, 0, 1, 0, 0, 0, 1): 74.0,
 (1, 0, 0, 1, 0, 0, 1, 1, 0): 76.0,
 (0, 1, 0, 0, 0, 0, 0, 1, 0): 77.0,
 (0, 0, 0, 1, 0, 0, 1, 0, 1): 78.0,
 (0, 1, 0, 0, 1, 0, 0, 1, 0): 79.0,
 (0, 1, 0, 1, 1, 0, 1, 0, 1): 82.0,
 (0, 0, 0, 1, 1, 1, 0, 0, 0): 83.0,
 (0, 1, 0, 0, 1, 0, 0, 0, 0): 83.0,
 (0, 1, 1, 0, 0, 0, 0, 1, 1): 85.66666666666667,
 (1, 0, 0, 0, 0, 1, 0, 0, 1): 88.25,
 (1, 0, 0, 0, 0, 1, 0, 0, 0): 89.0,
 (0, 0, 1, 1, 0, 0, 0, 0, 0): 89.0,
 (1, 1, 0, 0, 0, 0, 0, 0, 0): 89.0,
 (0, 0, 0, 0, 0, 0, 1, 0, 1): 89.0,
 (0, 0, 0, 0, 1, 1, 0, 0, 0): 89.0,
 (0, 0, 0, 0, 0, 0, 0, 1, 1): 89.0,
 (0, 0, 0, 1, 1, 0, 0, 0, 0): 89.0,
 (0, 1, 1, 0, 1, 1, 0, 1, 1): 92.8,
 (0, 0, 0, 1, 1, 1, 0, 0, 1): 93.0,
 (1, 1, 0, 0, 1, 0, 1, 1, 0): 95.0,
 (0, 0, 0, 0, 0, 0, 0, 0, 1): 95.0,
 (1, 0, 0, 0, 0